In [1]:
# Import libraries
import sys

sys.path.append("..")
from src.preprocessing import get_transforms, get_datasets, get_dataloaders

import torch
import matplotlib.pyplot as plt
from src.config import config
from src.model import SegFormer3D

# Set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

ModuleNotFoundError: No module named 'custom_transforms'

In [ ]:
# Get the transforms
transform = get_transforms(
    resize_shape=config["resize_shape"], contrast_value=config["contrast_value"]
)

# Get the datasets
train_dataset, val_dataset = get_datasets(
    root_dir="../data",
    collection="HCC-TACE-Seg",
    transform=transform,
    download=True,
    download_len=5,
    seg_type="SEG",
    val_frac=config["val_frac"],
    seed=config["seed"],
)

# Get the dataloaders
train_loader, val_loader = get_dataloaders(
    train_dataset,
    val_dataset,
    batch_size=config["batch_size"],
    num_workers=config["num_workers"],
)

In [ ]:
# Sample a batch of data from the dataloader
batch = next(iter(train_loader))

# Separate the image and segmentation from the batch
image, seg = batch["image"], batch["seg"][:, 2]

print(image.shape, seg.shape)

In [ ]:
# Evaluate the model
model = SegFormer3D(num_classes=config["num_classes"])
model.load_state_dict(torch.load("../models/model.pth"))
model.eval()

# Move the model to the device
model = model.to(device)

In [ ]:
# Set a evaluation script
outputs = model(image.to(device))
prediction = (torch.sigmoid(outputs) > 0.5).float()

print(prediction.shape)
print(prediction.unique())

In [ ]:
import nibabel as nib
import numpy as np

print(prediction.shape, seg.shape)

# Check if the first dimension is 1 before squeezing
prediction = prediction.squeeze((0, 1))

# Convert to numpy array and change dtype to float32
prediction_numpy = prediction.cpu().numpy().astype(np.float32)

# Save the segmentation target
if seg.size(0) == 1:
    seg = seg.squeeze(0)

seg_numpy = seg.cpu().numpy().astype(np.float32)


In [ ]:
# Check the unique values and their count
print(np.unique(prediction_numpy, return_counts=True))

print(np.unique(seg_numpy, return_counts=True))

difference = prediction_numpy - seg_numpy
num_differing_voxels = np.count_nonzero(difference)
print(f"Number of differing voxels: {num_differing_voxels}")

In [ ]:
# Save prediction and target volumes as nifti files
new_image = nib.Nifti1Image(prediction_numpy, np.eye(4))
nib.save(new_image, "prediction.nii")

new_image = nib.Nifti1Image(seg_numpy, np.eye(4))
nib.save(new_image, "target.nii")